In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_x = train_data.iloc[:,2:24]
train_y = train_data.iloc[:,24]
test_x = test_data.iloc[:,2:24]

In [ ]:
test_x.shape

(19919, 22)

In [ ]:
train_x.shape

(36992, 22)

In [ ]:
df = pd.concat([test_x.assign(ind="test"), train_x.assign(ind="train")])

In [ ]:
df.isnull().sum()

age                                0
gender                             0
security_no                        0
region_category                 8376
membership_category                0
joining_date                       0
joined_through_referral            0
referral_id                        0
preferred_offer_types            447
medium_of_operation                0
internet_option                    0
last_visit_time                    0
days_since_last_login              0
avg_time_spent                     0
avg_transaction_value              0
avg_frequency_login_days           0
points_in_wallet                5406
used_special_discount              0
offer_application_preference       0
past_complaint                     0
complaint_status                   0
feedback                           0
ind                                0
dtype: int64

In [ ]:
for i in df.columns:
  print(i,' :' ,len(df[i].unique()) ,' unique labels')

age  : 55  unique labels
gender  : 3  unique labels
security_no  : 56911  unique labels
region_category  : 4  unique labels
membership_category  : 6  unique labels
joining_date  : 1096  unique labels
joined_through_referral  : 3  unique labels
referral_id  : 14369  unique labels
preferred_offer_types  : 4  unique labels
medium_of_operation  : 4  unique labels
internet_option  : 3  unique labels
last_visit_time  : 41745  unique labels
days_since_last_login  : 27  unique labels
avg_time_spent  : 35745  unique labels
avg_transaction_value  : 56637  unique labels
avg_frequency_login_days  : 2569  unique labels
points_in_wallet  : 31474  unique labels
used_special_discount  : 2  unique labels
offer_application_preference  : 2  unique labels
past_complaint  : 2  unique labels
complaint_status  : 5  unique labels
feedback  : 9  unique labels
ind  : 2  unique labels


In [ ]:
df.drop(['security_no','joining_date','referral_id','last_visit_time','avg_frequency_login_days'], axis=1, inplace=True)

In [ ]:
df['region_category'].fillna("None",inplace=True)

In [ ]:
df['preferred_offer_types'].fillna("Without Offers",inplace=True)

In [ ]:
df['points_in_wallet'] = df['points_in_wallet'].fillna(df['points_in_wallet'].mean(axis=0))

In [ ]:
df.head()

,age,gender,region_category,membership_category,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,days_since_last_login,avg_time_spent,avg_transaction_value,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,ind
0,50,F,Village,Premium Membership,No,Without Offers,Smartphone,Wi-Fi,12,386.26,40721.44,733.830000,Yes,No,No,Not Applicable,Poor Product Quality,test
1,41,M,Village,Gold Membership,No,Without Offers,Desktop,Fiber_Optic,11,37.80,9644.40,726.000000,Yes,No,No,Not Applicable,Poor Website,test
2,31,F,Town,Silver Membership,No,Gift Vouchers/Coupons,Both,Mobile_Data,18,215.36,3693.25,713.780000,Yes,No,Yes,Solved in Follow-up,No reason specified,test
3,64,M,Town,Silver Membership,Yes,Credit/Debit Card Offers,?,Fiber_Optic,-999,44.57,36809.56,744.970000,Yes,No,Yes,No Information Available,Too many ads,test
4,16,F,Town,No Membership,Yes,Without Offers,Smartphone,Mobile_Data,6,349.88,40675.86,299.048351,No,Yes,Yes,Solved in Follow-up,Poor Website,test


In [ ]:
df.dtypes

age                               int64
gender                           object
region_category                  object
membership_category              object
joined_through_referral          object
preferred_offer_types            object
medium_of_operation              object
internet_option                  object
days_since_last_login             int64
avg_time_spent                  float64
avg_transaction_value           float64
points_in_wallet                float64
used_special_discount            object
offer_application_preference     object
past_complaint                   object
complaint_status                 object
feedback                         object
ind                              object
dtype: object

In [ ]:
test_x, train_x = df[df["ind"].eq("test")], df[df["ind"].eq("train")]
train_x.drop('ind', axis=1, inplace=True)
test_x.drop('ind', axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
obj = (train_x.dtypes == 'object')
object_cols = list(obj[obj].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['gender', 'region_category', 'membership_category', 'joined_through_referral', 'preferred_offer_types', 'medium_of_operation', 'internet_option', 'used_special_discount', 'offer_application_preference', 'past_complaint', 'complaint_status', 'feedback']


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
label_X_train = train_x.copy()
label_X_test = test_x.copy()
for col in object_cols:
  label_X_train[col] = encoder.fit_transform(train_x[col])
  label_X_test[col] = encoder.transform(test_x[col])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=120)
model.fit(label_X_train,train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=120,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
y_pred = model.predict(label_X_test)

In [ ]:
y_pred

array([4, 4, 4, ..., 4, 4, 4])

In [ ]:
output = pd.DataFrame({'customer_id':test_data['customer_id'], 'churn_risk_score':y_pred})
output.to_csv('submission.csv', index=False)